In [2]:
from helpers.helpers_python.database_class import Database
import pandas as pd
import configparser

In [3]:
config = configparser.ConfigParser()
config.read_file(open('dl.cfg'))

aws_access_key_id = config['AWS']['key'] 
aws_secret_access_key = config['AWS']['secret']
host = config['REDSHIFT_CLUSTER']['db_host']
port = config['REDSHIFT_CLUSTER']['db_port']
dbname = config['REDSHIFT_CLUSTER']['db_name']
user = config['REDSHIFT_CLUSTER']['db_user']
password = config['REDSHIFT_CLUSTER']['db_password']

In [4]:
db = Database(host, dbname, port, user, password)
db.connect()     

In [19]:
query_crime_location_data_agg = """
        select
              hlm.location_name
            , count(distinct sk_crime_id) as crimes
        from public.public.f_uk_crime_events fuce
        left join public.public.d_investigation_outcome dio on fuce.investigation_outcome_id = dio.investigation_outcome_id
        left join public.public.d_location dl on fuce.location_id = dl.location_id
        left join public.public.h_location_mapping hlm on dl.lsoa_code = hlm.lsoa_code 
        group by 1 order by 2 desc 
        limit 10
"""



query_crime_outcome_data_agg = """
        select
              dio.investigation_outcome_category
            , count(distinct sk_crime_id) as crimes
        from public.public.f_uk_crime_events fuce
        inner join public.public.d_investigation_outcome dio 
            on fuce.investigation_outcome_id = dio.investigation_outcome_id 
                and dio.investigation_outcome_category != 'nan'
        left join public.public.d_location dl on fuce.location_id = dl.location_id
        left join public.public.h_location_mapping hlm on dl.lsoa_code = hlm.lsoa_code 
        group by 1 order by 2 desc 
"""


query_crime_yearly = """
        select
              date_part(year, to_date(fuce.event_month,'yyyy-mm')) as year
            , count(distinct sk_crime_id) as crimes
        from public.public.f_uk_crime_events fuce
        group by 1 order by 1 asc
"""

In [16]:
df = db.query_to_df(sql=query_crime_location_data_agg)
df

,location_name,crimes
0,Birmingham,332197
1,Leeds,327969
2,Bradford,232622
3,Liverpool,207303
4,Westminster,188968
5,Sheffield,186473
6,County Durham,177468
7,"Bristol, City of",163704
8,Nottingham,147013
9,Kirklees,146935


In [20]:
df = db.query_to_df(sql=query_crime_outcome_data_agg)
df

,investigation_outcome_category,crimes
0,"Investigation complete, no suspect identified",5127386
1,Unable to prosecute suspect,4193171
2,Status update unavailable,1763640
3,Court result unavailable,709634
4,Under investigation,625316
5,Local resolution,315781
6,Awaiting court outcome,161685
7,Offender given a caution,155654
8,Further investigation is not in the public int...,144435
9,Action to be taken by another organisation,134183


In [18]:
df = db.query_to_df(sql=query_crime_yearly)
df

,year,crimes
0,2018.0,1458191
1,2019.0,5716337
2,2020.0,5384524
3,2021.0,3849336
